In [65]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 sentence-transformers langchain -groq --quiet


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -g


In [66]:
!pip list | grep crewai_tools

In [67]:
!pip install crewai
!pip install 'crewai[tools]'

In [ ]:
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools import tool
from crewai import Crew
from crewai import Task
from crewai import Agent

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
os.environ['GROQ_API_KEY'] = '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'

In [ ]:
llm = ChatOpenAI(
    openai_api_base = "https://api.groq.com/openai/v1",
    openai_api_key = os.environ['GROQ_API_KEY'],
    model_name = "llama3-8b-8192",
    temperature = 0.1,
    max_tokens=1000,
)


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.8 MB/s eta 0:00:00


MERGING AND EXTRACTING PDF

In [ ]:
from crewai_tools.tools.pdf_search_tool.pdf_search_tool import PDFSearchTool


In [71]:
!pip install PyPDF2

In [75]:
import requests
from PyPDF2 import PdfMerger

# List of PDF URLs
pdf_urls = [
    '#',
    '#add your pdf links over here',
    '#add the pdf links',
    '#pdf links goes here'

]

# Initialize a PdfMerger object
merger = PdfMerger()

# Function to download PDFs and merge them
def download_and_merge_pdfs(urls, output_filename):
    for i, pdf_url in enumerate(urls):
        response = requests.get(pdf_url)
        temp_pdf = f'temp_document_{i+1}.pdf'  # Temporary filename for each PDF
        with open(temp_pdf, 'wb') as f:
            f.write(response.content)
        merger.append(temp_pdf)  # Add the PDF to the merger

    # Write out the merged PDF
    with open(output_filename, 'wb') as f_out:
        merger.write(f_out)

    merger.close()
    print(f'All PDFs merged into: {output_filename}')

# Merge all PDFs into a single file
download_and_merge_pdfs(pdf_urls, 'merged_document.pdf')


All PDFs merged into: merged_document.pdf


In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install langchain_huggingface

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install langchain_openai

In [76]:
rag_tool = PDFSearchTool(pdf='merged_document.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                #temperature=0.5,
                #top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                # task_type="retrieval_document",
                #title="Embeddings",
            ),
        ),
    )
)

Inserting batches in chromadb: 100%|██████████| 4/4 [01:25<00:00, 21.43s/it]


In [126]:
response = rag_tool.run("what are  Key Role and Responsibilities: Buyers in GeM Stakeholders Ecosystem?")
print(response)


Using Tool: Search a PDF's content
Relevant Content:
1. Introduction 1.1 GeM Charter 1.2 GeM Special Purpose Vehicle 1.3 Founding Principles of the GeM Initiative 1.4 Objective of this Document 2. GeM Platform 2.1 Scope of Operations for the GeM Platform 2.2 Potential for the GeM Platform 2.3 Key Features Supported by the GeM Platform 2.4 GeM Procurement Guidelines 2.5 Monetization on the GeM Platform 3. GeM Organization (GeM SPV) 3.1 Vision, Mission and Values for GeM SPV 3.2 Organization Structure 3.3 Functioning of the GeM SPV 3.4 Governance Structure 4. GeM Stakeholders Ecosystem 4.1 Key Role and Responsibilities: Buyers 4.2 Key Role and Responsibilities: Sellers 4.3 Interfacing with Other Key Stakeholders: Guiding Principles of Collaboration 5. Appendix 5.1 Appendix A – Templates for GeM scoreboards 5.2 Appendix B - GeM Procurement Guidelines6 6 7 8 9 10 10 11 11 19 21 23 23 24 25 27 28 28 29 29 32 32 33TABLE OF CONTENT

the Government. 1.3.2 Role and Nature of the GeM Initiative 

In [81]:
!pip install PyPDF2

In [82]:
os.environ['TAVILY_API_KEY'] = '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'

In [83]:
web_search_tool = TavilySearchResults( k=3)

In [127]:
web_search_tool.run("what are  Key Role and Responsibilities: Buyers in GeM Stakeholders Ecosystem?")

[{'url': 'https://assets-bg.gem.gov.in/resources/upload/shared_doc/jd-buyer-management-officer-defence-integration_1714372439.pdf',
  'content': 'Job Summary: This multifaceted role demands meticulous attention to detail, proactive engagement with stakeholders, and a deep understanding of defence procurement dynamics within the framework of GeM. This job description outlines the key responsibilities, qualifications, and skills required for the role of Buyer Management Officer (Defence Integration), Defence Integration on the Government ...'},
 {'url': 'https://assets-bg.gem.gov.in/resources/upload/shared_doc/jd-manager-category-coordination-management_1714549792.pdf',
  'content': 'GeM provides a paperless, cashless, and contactless ecosystem for government buyers to directly purchase products and services from pan-India sellers and service providers through an online platform. GeM covers the entire gamut of procurement process, right from vendor registration and item selection by buye

TRYING FOR BETTER ROUTER TOOL

In [128]:
@tool
def router_tool(question: str) -> str:
    """Router Function to route the question to the appropriate tool based on keywords."""
    question = question.lower()  # Normalize the question to lowercase for consistent matching

    # Define routing criteria
    if any(keyword in question for keyword in ['vector', 'embedding']):
        return 'embedder'
    elif any(keyword in question for keyword in ['search', 'find']):
        return 'web_search'
    elif any(keyword in question for keyword in ['analysis', 'extraction']):
        return 'analysis_tool'
    else:
        return 'default_tool'  # Default tool for questions that don't match specific criteria



In [129]:
#Router_agent

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [103]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [104]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [105]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [106]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [117]:
##task

router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [118]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [119]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [120]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)




In [121]:
answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

In [122]:
#Define the flow for our usecase
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

In [130]:
inputs ={"question":"what are  Key Role and Responsibilities: Buyers in GeM Stakeholders Ecosystem?"}

In [131]:
result = rag_crew.kickoff(inputs)

 [2024-08-22 06:25:22][DEBUG]: == Working Agent: Router
 [2024-08-22 06:25:22][INFO]: == Starting Task: Analyse the keywords in the question what are  Key Role and Responsibilities: Buyers in GeM Stakeholders Ecosystem?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: I need to analyze the keywords in the question to determine whether it's eligible for a vectorstore search or a web search.

Action: router_tool
Action Input: {'question': {'title': 'What are Key Role and Responsibilities: Buyers in GeM Stakeholders Ecosystem?', 'type': 'string'}}
 

I encountered an error while trying to use the tool. This was the error: 1 validation error for Router_Tool
question
  str type expected (type=type_error.str

In [132]:
print(result)

The key roles and responsibilities of buyers in the GeM stakeholders ecosystem are:

* Placing orders and making payments through the GeM platform
* Providing necessary information and documents for procurement
* Monitoring the status of their orders and resolving any issues that may arise
* Providing feedback to the GeM platform and other stakeholders
* Ensuring compliance with GeM's terms and conditions and other relevant regulations

Note: The above answer is based on the assumption that the hallucination task has provided a relevant and useful response.


In [141]:
input1={"question":" what is full form of DMSRDE?"}

In [142]:
result1 = rag_crew.kickoff(input1)

 [2024-08-22 06:37:46][DEBUG]: == Working Agent: Router
 [2024-08-22 06:37:46][INFO]: == Starting Task: Analyse the keywords in the question  what is full form of DMSRDE?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: I need to analyze the keywords in the question to determine whether it's eligible for a vectorstore search or a web search.

Action: router_tool
Action Input: {'question': {'title': 'what is full form of DMSRDE', 'type': 'string'}}
 

I encountered an error while trying to use the tool. This was the error: 1 validation error for Router_Tool
question
  str type expected (type=type_error.str).
 Tool router_tool accepts these inputs: router_tool(question: 'string') - Router Function to rou

In [143]:
print(result1)

Please wait while I perform the web search...

After searching the web, I found the following answer:

DMSRDE stands for Defence Materials and Stores Research and Development Establishment.
